In [40]:
import sys
import os

# Get the absolute path to the project root directory
# This works if your notebook is in a notebooks/ directory one level below the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit
from src import clean_data

## Input Processing

Import the file skipping the first 18 row
and then select the column and transpose

In [42]:
file_name = "../Data/RLC_time.csv"
data = np.genfromtxt(file_name, skip_header=18, delimiter=",")

for element in data[:10]:
    print(element)

[      nan       nan       nan  2.16e-06 -2.24e-01       nan]
[      nan       nan       nan  2.18e-06 -2.08e-01       nan]
[     nan      nan      nan  2.2e-06 -2.0e-01      nan]
[      nan       nan       nan  2.22e-06 -1.76e-01       nan]
[      nan       nan       nan  2.24e-06 -1.60e-01       nan]
[      nan       nan       nan  2.26e-06 -1.44e-01       nan]
[      nan       nan       nan  2.28e-06 -1.36e-01       nan]
[     nan      nan      nan  2.3e-06 -1.2e-01      nan]
[      nan       nan       nan  2.32e-06 -1.04e-01       nan]
[      nan       nan       nan  2.34e-06 -8.80e-02       nan]


In [43]:
x_data, y_data = clean_data(data)
for x ,y in tuple(zip(x_data,y_data)):
    print(x,y)

2.16e-06 -0.224
2.18e-06 -0.208
2.2e-06 -0.2
2.22e-06 -0.176
2.24e-06 -0.16
2.26e-06 -0.144
2.28e-06 -0.136
2.3e-06 -0.12
2.32e-06 -0.104
2.34e-06 -0.088
2.36e-06 -0.072
2.38e-06 -0.056
2.4e-06 -0.04
2.42e-06 -0.024
2.44e-06 -0.008
2.46e-06 0.008
2.48e-06 0.024
2.5e-06 0.032
2.52e-06 0.048
2.54e-06 0.064
2.56e-06 0.08
2.58e-06 0.096
2.6e-06 0.104
2.62e-06 0.12
2.64e-06 0.136
2.66e-06 0.152
2.68e-06 0.168
2.7e-06 0.184
2.72e-06 0.2
2.74e-06 0.208
2.76e-06 0.224
2.78e-06 0.24
2.8e-06 0.256
2.82e-06 0.264
2.84e-06 0.28
2.86e-06 0.296
2.88e-06 0.312
2.9e-06 0.32
2.92e-06 0.336
2.94e-06 0.344
2.96e-06 0.36
2.98e-06 0.376
3e-06 0.384
3.02e-06 0.4
3.04e-06 0.408
3.06e-06 0.424
3.08e-06 0.44
3.1e-06 0.448
3.12e-06 0.464
3.14e-06 0.472
3.16e-06 0.488
3.18e-06 0.504
3.2e-06 0.512
3.22e-06 0.52
3.24e-06 0.536
3.26e-06 0.544
3.28e-06 0.552
3.3e-06 0.568
3.32e-06 0.584
3.34e-06 0.592
3.36e-06 0.6
3.38e-06 0.608
3.4e-06 0.616
3.42e-06 0.632
3.44e-06 0.64
3.46e-06 0.656
3.48e-06 0.664
3.5e-06 0.672
3